<pre>
*****PySpark DF Question*****
Filename: txns
Metadata: txnno: String, txndate: String, custno: String, amount: String, category: String, product: String, city: String, state: String, spendby: String
1) Read the file & show 5 records.
2) Separate txndate in date, month, year Column. Convert date column into words (e.g. 01 --> Sunday, 02 --> Monday).
3) Find the sum of amount daywise.
4) Write the output data in json format.
Output Columns:
Date(Words),Sum(Amount)

Create a DAG in Airflow & with spark-submit execute it.
</pre>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("yettodecide").master("local").getOrCreate()

In [15]:
df = spark.read.format("csv")\
                .option("header",True)\
                .option("InferSchema",True)\
                .load("C:\\Personal\\Projects\\Spark\\dataset\\txns")
df.show(5,truncate=False)

+-----+----------+-------+------+------------------+---------------------------------+-----------+----------+-------+
|txnno|txndate   |custno |amount|category          |product                          |city       |state     |spendby|
+-----+----------+-------+------+------------------+---------------------------------+-----------+----------+-------+
|0    |06-26-2011|4007024|40.33 |Exercise & Fitness|Cardio Machine Accessories       |Clarksville|Tennessee |credit |
|1    |05-26-2011|4006742|198.44|Exercise & Fitness|Weightlifting Gloves             |Long Beach |California|credit |
|2    |06-01-2011|4009775|5.58  |Exercise & Fitness|Weightlifting Machine Accessories|Anaheim    |California|credit |
|3    |06-05-2011|4002199|198.19|Gymnastics        |Gymnastics Rings                 |Milwaukee  |Wisconsin |credit |
|4    |12-17-2011|4002613|98.81 |Team Sports       |Field Hockey                     |Nashville  |Tennessee |credit |
+-----+----------+-------+------+------------------+----

In [16]:
from pyspark.sql.functions import col, year, month, date_format, to_timestamp

In [17]:
df = df.select(col("*"),
      year(to_timestamp(col("txndate"),"MM-dd-yyyy")).alias("Year"),
      month(to_timestamp(col("txndate"),"MM-dd-yyyy")).alias("Year"),
      date_format(to_timestamp(col("txndate"),"MM-dd-yyyy"),"EEEE").alias("Day")
)

In [18]:
df.show(5)

+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+----+----+---------+
|txnno|   txndate| custno|amount|          category|             product|       city|     state|spendby|Year|Year|      Day|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+----+----+---------+
|    0|06-26-2011|4007024| 40.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee| credit|2011|   6|   Sunday|
|    1|05-26-2011|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California| credit|2011|   5| Thursday|
|    2|06-01-2011|4009775|  5.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California| credit|2011|   6|Wednesday|
|    3|06-05-2011|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin| credit|2011|   6|   Sunday|
|    4|12-17-2011|4002613| 98.81|       Team Sports|        Field Hockey|Nashville  | Tennessee| credit|2011|  12| Saturday|


In [23]:
df2 = df["txndate","amount"].groupby("txndate").sum()

In [24]:
df2.write.format('csv')\
         .option("header",True)\
         .mode("overwrite")\
         .save("C:\\Personal\\Projects\\Spark\\dataset\\6th_assignment.csv")